# **IEOR 4571 Fall 2020 Homework2 Report**


- Hu, Bo (uni: bh2569)
- Qin, Rui (uni: rq217)
- Yuan, Shuibenyang (uni: sy2938)

In [3]:
import sys
sys.path.append('../')

# Objective

# The Sample

In [4]:
# todo how did we sample the data

In [45]:
from src.utils import loading, Spark
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pandas as pd
import numpy as np

## Load Sample

In [6]:
# create spark session
spark = Spark()

Spark UI address http://127.0.0.1:4040


In [12]:
# load sample data from '/data/raw/sample.csv'
datas = loading(spark, '../data/raw')
sample = datas['sample']

## Sample Objectives

In [82]:
print(f'''
            number of data points in the sample: {sample.count()},
            number of unique users in the sample: {sample.select('userId').distinct().count()},
            number of unique movies in the sample: {sample.select('movieId').distinct().count()},
            number of movies a user rated:{sample.groupby('userId').agg(F.count('movieId').alias('cnt')).select(F.mean('cnt')).collect()[0][0]},
            number of users a movie be rated: {sample.groupby('movieId').agg(F.count('userId').alias('cnt')).select(F.mean('cnt')).collect()[0][0]},
            average rating: {sample.select(F.mean('rating')).collect()[0][0]},
            standard deviation of rating: {sample.select(F.stddev('rating')).collect()[0][0]},
            average rating by user: {sample.groupby('userId').agg(F.mean('rating').alias('rating')).select(F.mean('rating')).collect()[0][0]},
            standard deviation of rating by user mean: {sample.groupby('userId').agg(F.mean('rating').alias('rating')).select(F.stddev('rating')).collect()[0][0]},
            average rating by movie: {sample.groupby('movieId').agg(F.mean('rating').alias('rating')).select(F.mean('rating')).collect()[0][0]},
            standard deviation of rating by movie mean: {sample.groupby('movieId').agg(F.mean('rating').alias('rating')).select(F.stddev('rating')).collect()[0][0]}
        ''')


            number of data points in the sample: 120319,
            number of unique users in the sample: 24468,
            number of unique movies in the sample: 1102,
            number of movies a user rated:4.917402321399379,
            number of users a movie be rated: 109.18239564428312,
            average rating: 3.5531462196328096,
            standard deviation of rating: 1.0083735197631891,
            average rating by user: 3.5953343581686648,
            standard deviation of rating by user mean: 0.6747330910068228,
            average rating by movie: 3.4874705002463724,
            standard deviation of rating by movie mean: 0.4306367931067497
        


# Evaluation

## Cross Validation Setup

In [83]:
from src.evaluation import Evaluator, Cross_validate_als
from src.model_based import Als

In [84]:
splits = loading(spark, '../data/interim')

In [97]:
print(list(splits.keys()))

['train_0.5_0.5', 'train_0.25_0.75', 'test_0.5_0.5', 'test_0.25_0.75', 'train_0.75_0.25', 'test_0.75_0.25']


In [100]:
list(evaluators.keys()) + ['train', 'test']

['rmse', 'accuracy', 'coverage_10', 'train', 'test']

In [86]:
# build evaluation pipeline
def evaluate(train, test, evaluators, model):
    model.fit(train)
    train_pred = model.predict(train)
    test_pred = model.predict(test)
    res = pd.DataFrame(np.zeros((len(evaluators) + 2, len(evaluators) + 2))
                       ,columns = list(evaluators.key()) + ['train'], ['test'])
    for eva in evaluators.keys():
        res['train_'+eva] = evaluators[eva].evaluate(train_pred)
        res['test_'+eva] = evaluators[eva].evaluate(test_pred)
    return res

## Metrics

In [101]:
len(evaluators)

3

In [87]:
evaluators = {'rmse': Evaluator(metrics = 'rmse'), 
              'accuracy': Evaluator(metrics = 'accuracy'), 
              'coverage_2': Evaluator(metrics = 'converage_k', 
                                       ratingCol='rating', 
                                       predCol='prediction', 
                                       idCol='userId', 
                                       k=2)}

## Memory Based Collaborative Filtering

In [88]:
from src.memory_based import Memory_based_CF

### User Based Collaborative Filtering

In [89]:
cf = Memory_based_CF(spark, 'user')

In [90]:
result = []
for i in ['0.75_0.25', '0.5_0.5', '0.25_0.75']:
    train, test = splits['train_' + i], splits['test_' + i]
    res = evaluate(train, test, evaluators, cf)
    result.append(res)

100%|██████████| 65446/65446 [00:47<00:00, 1364.35it/s]


In [94]:
result[0]

{'train_rmse': 0.7588023855356845,
 'test_rmse': 1.159159071949617,
 'train_accuracy': 0.7873901108060949,
 'test_accuracy': 0.5818169262111115,
 'train_coverage_10': 0.6473895063323857,
 'test_coverage_10': 0.29509962304792675}

In [93]:
pd.DataFrame(result[0])

ValueError: If using all scalar values, you must pass an index

### Item Based Collaborative Filtering

## Model Based Collaborative Filtering